In [1]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 6.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import mysql.connector
import os

# Load the CSV file into a pandas DataFrame
file_path = 'train.csv'
df = pd.read_csv(file_path)

# Extract the file name (without extension) to use as table name
table_name = os.path.splitext(os.path.basename(file_path))[0]

# Clean the data, replacing 'NA' or NaN values with None (which will map to NULL in MySQL)
df = df.where(pd.notnull(df), None)

# Connect to MySQL database
connection = mysql.connector.connect(
    host='mysql',  # Your MySQL host
    user='haiduong',  # Your MySQL username
    password='haiduong',  # Your MySQL password
    database='tlcn'  # Your database name
)

cursor = connection.cursor()

def get_mysql_type(column):
    if pd.api.types.is_integer_dtype(column):
        return "INT"
    elif pd.api.types.is_float_dtype(column):
        return "FLOAT"
    elif pd.api.types.is_object_dtype(column):
        return "VARCHAR(255)"  # Adjust VARCHAR size as needed
    else:
        return "TEXT"  # Default type for other types

# Prepare a CREATE TABLE statement dynamically
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"

# Loop through the columns in the DataFrame to define column types
for i, col in enumerate(df.columns):
    mysql_type = get_mysql_type(df[col])
    if i == 0:  # Assuming the first column should be the primary key
        create_table_query += f"  `{col}` {mysql_type} PRIMARY KEY,\n"
    else:
        create_table_query += f"  `{col}` {mysql_type},\n"

# Remove the last comma and close the query
create_table_query = create_table_query.rstrip(',\n') + "\n);"

# Execute the CREATE TABLE query
cursor.execute(create_table_query)
connection.commit()

# Prepare an INSERT query dynamically
columns = ", ".join([f"`{col}`" for col in df.columns])
placeholders = ", ".join(["%s"] * len(df.columns))
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Insert data row by row
for index, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("inserted successfully!")


inserted successfully!
